<a href="https://colab.research.google.com/github/aerau05/ds3001proj/blob/main/ds3001Proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# loaded DEMO.xpt and linked_mortality_file_1999_2000.csv (both in the repo) into colab
# then just used provided code segment from assignment
import pandas as pd
mdf = pd.read_csv('linked_mortality_file_1999_2000.csv') # Load mortality file
print( mdf.head() )
gdf = pd.read_sas("DEMO.xpt", format="xport") # Load demographics file
print( gdf.head() )
df = gdf.merge(mdf, on="SEQN", how="inner") # Merge mortality and demographics on SEQN variable

   SEQN  ELIGSTAT  MORTSTAT  UCOD_LEADING  DIABETES  HYPERTEN  PERMTH_INT  \
0     1         2       NaN           NaN       NaN       NaN         NaN   
1     2         1       1.0           6.0       0.0       0.0       177.0   
2     3         2       NaN           NaN       NaN       NaN         NaN   
3     4         2       NaN           NaN       NaN       NaN         NaN   
4     5         1       0.0           NaN       NaN       NaN       244.0   

   PERMTH_EXM  
0         NaN  
1       177.0  
2         NaN  
3         NaN  
4       244.0  
   SEQN  SDDSRVYR  RIDSTATR  RIDEXMON  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDAGEEX  \
0   1.0       1.0       2.0       2.0       2.0       2.0      29.0      31.0   
1   2.0       1.0       2.0       2.0       1.0      77.0     926.0     926.0   
2   3.0       1.0       2.0       1.0       2.0      10.0     125.0     126.0   
3   4.0       1.0       2.0       2.0       1.0       1.0      22.0      23.0   
4   5.0       1.0       2.0       2.

ELIGSTAT indicates whether a NHANES particpant was eligible to be linked to the National Dealth Index for mortality follow-up. Individuals who are classified as eligible have suffiecient identifying information (like name, birth date, and SSN) to allow accurate matching to death records. Participants who are ineligilbe and cannot be reliably followed for mortality outcomes and therefore are excluded from certain analses. This variable is important because only eligible can be used to study survival and death outcomes.

The MORSTAT variable represensts the participants mortalitiy status at the end of the follow-up period. It is a binary variable indicating whether the individual was alive or deceased. A value of 0 typicall means the participant was alive at the end of the follow-up, while a value of 1 indicates that the participant had died. This variable serves as the primary outcome variable for predicting mortality and is central to analyses that attempt to model clear or classify death risk using demographic, behavioral, and health data.

PERMTH_INT measures the total number of months each participant was followed from the date of their NHANES examination until either death or the end of the mortality follow-up period. This variable provides a measrue of survival time and can be interpreted as the length of time a person remained alive after being surveyed. It is useful for studying life expectancy and time-to-event outcomes and allows researchers to model not only whether someone died, but how long they survived.

RIDAGEEX represents the participant's age in years at the time of the NHANES medical examination. Age is one of the strongest predictors of mortatlity and is a critical demosgraphic variable in health and survival analysis. This variable allows researchers to account for differences in mortality risk across age groups and serves as an essential covariate when predicting both mortality status and life expectancy.

In [26]:
smq = pd.read_sas("SMQ.xpt", format="xport")  # Smoking
bmx = pd.read_sas("BMX.xpt", format="xport")  # Body measures
bpx = pd.read_sas("BPX.xpt", format="xport")  # Blood pressure
diq = pd.read_sas("DIQ.xpt", format="xport")  # Diabetes

print("SMQ shape:", smq.shape)
print("BMX shape:", bmx.shape)
print("BPX shape:", bpx.shape)
print("DIQ shape:", diq.shape)

# Merge all datasets on SEQN
df = df.merge(smq, on="SEQN", how="left")
print("After merging SMQ:", df.shape)
df = df.merge(bmx, on="SEQN", how="left")
print("After merging BMX:", df.shape)
df = df.merge(bpx, on="SEQN", how="left")
print("After merging BPX:", df.shape)
df = df.merge(diq, on="SEQN", how="left")
print("After merging DIQ:", df.shape)

# preview final dataset and missing values
print(df.head())
missing_counts = df.isna().sum()
missing_percent = df.isna().mean() * 100
missing_summary = pd.DataFrame({
    "Missing_Count": missing_counts,
    "Missing_Percent": missing_percent
}).sort_values(by="Missing_Percent", ascending=False)
print(missing_summary.head(20))

SMQ shape: (4880, 39)
BMX shape: (9282, 38)
BPX shape: (9282, 30)
DIQ shape: (9493, 17)
After merging SMQ: (9965, 429)
After merging BMX: (9965, 466)
After merging BPX: (9965, 495)
After merging DIQ: (9965, 511)
   SEQN  SDDSRVYR  RIDSTATR  RIDEXMON  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDAGEEX  \
0   1.0       1.0       2.0       2.0       2.0       2.0      29.0      31.0   
1   2.0       1.0       2.0       2.0       1.0      77.0     926.0     926.0   
2   3.0       1.0       2.0       1.0       2.0      10.0     125.0     126.0   
3   4.0       1.0       2.0       2.0       1.0       1.0      22.0      23.0   
4   5.0       1.0       2.0       2.0       1.0      49.0     597.0     597.0   

   RIDRETH1  RIDRETH2  ...  DIQ060U  DIQ070  DIQ080  DIQ090  DIQ100  DIQ110  \
0       4.0       2.0  ...      NaN     NaN     NaN     NaN     NaN     NaN   
1       3.0       1.0  ...      NaN     NaN     NaN     2.0     2.0     NaN   
2       3.0       1.0  ...      NaN     NaN     NaN     NaN     

To predict mortaltity status (MORSTAT) and survival time (PERMTH_INT), we selected NHANES variables that represent key demographic, behavorial, and health-related risk factors known to influence long-term health outcomes. The behavioral variable, smoaking status, was chosen because lifestyle behaviors are well-establised contributers to chronic diseases including cardiovascular diease, cancer, and respiratory illness, which are leading causes of deaht according to the World Health Organizaiton. Clinical and physiological measures such as mody mass index, blood pressure, and diabetes were included to capture underlying health conditions that directly affect survival outcomes. Elevated BMI and hypertension are recognized as risk factors for heart disease and stroke, while diabetes dignificantly increases the risk of premature mortaltiy according to the CDC. These datasets were selected because they provide standardized, population-representative measurements collected using consistent protocols. Together, this combination of demographic, behavioral,adn clinical variables provides a comprehensive and scientifically justified set of predictors for modeling mortality and survival time.  